In [1]:
import os 
import glob
import xarray as xr
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.models import *
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.utils import *
from tensorflow.python.keras import backend as K
import pickle
import copy
import gc


2022-04-25 09:20:55.256802: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


## Load NWP and observation data

In [3]:
###############################################################################
# load data
leadtimelist = ['00','01','02','03','04','05','06', \
    '07','08','09','10','11','12', \
    '13','14','15','16','17','18', \
    '19','20','21','22','23','24']
leadhour = 1
leadtime = leadtimelist[leadhour]
prefix = '/p/scratch/deepacf/deeprain/ji4/Beijing_Unet/FCNdata'
save_dir = os.path.join(prefix,'Results',leadtime,'exp27')
os.makedirs(save_dir,exist_ok=True)

filepath = os.path.join(prefix,'PT_vars_Train.nc')
with xr.open_dataset(filepath) as dfile:
    PT_vars_Train = np.array(dfile['PT_vars_Train'])
    NWP_time_Train = dfile['init_time']
    print(NWP_time_Train)

filepath = os.path.join(prefix,'GT_prep_Train.nc')
with xr.open_dataset(filepath) as dfile:
    GT_prep_Train = np.array(dfile['GT_prep_Train'])

filepath = os.path.join(prefix,'PT_vars_Test.nc')
with xr.open_dataset(filepath) as dfile:
    PT_vars_Test = np.array(dfile['PT_vars_Test'])
    NWP_time_Test = dfile['init_time']
    print(NWP_time_Test)
    lat = dfile['lat']
    lon = dfile['lon']
    n_lat = len(lat)
    n_lon = len(lon)

filepath = os.path.join(prefix,'GT_prep_Test.nc')
with xr.open_dataset(filepath) as dfile:
    GT_prep_Test = np.array(dfile['GT_prep_Test'])
###############################################################################


<xarray.DataArray 'init_time' (init_time: 1420)>
array([18040100., 18040106., 18040112., ..., 19093006., 19093012., 19093018.])
Coordinates:
    time       float64 ...
  * init_time  (init_time) float64 1.804e+07 1.804e+07 ... 1.909e+07 1.909e+07
    lead_hour  int32 ...
<xarray.DataArray 'init_time' (init_time: 724)>
array([20040100., 20040106., 20040112., ..., 20093006., 20093012., 20093018.])
Coordinates:
    time       float64 ...
  * init_time  (init_time) float64 2.004e+07 2.004e+07 ... 2.009e+07 2.009e+07
    lead_hour  int32 ...


In [5]:
# model forecast
NWP_fcst_ids = 28
Mtest = copy.deepcopy(PT_vars_Test[:,:,:,NWP_fcst_ids:NWP_fcst_ids+1])
Ytest = copy.deepcopy(GT_prep_Test)


## Data preprocessing

In [9]:
###############################################################################
# normalization  加0.01，取LOG,是的预报出来的值没有0
def Scaler(k, array):
    array[np.where(array<0)] = 0
    return np.log(array+k)-np.log(k)

def invScaler(k, array):    
    return np.exp(array+np.log(k))-k

def NormMaxMin(array,dim):
    array_max = np.nanmax(array,dim)
    array_min = np.nanmin(array,dim)
    array_nor = (array-array_min)/(array_max-array_min)
    return array_nor,array_max,array_min

def invNormMaxMin(array_nor,array_max,array_min):
    array = array_nor*(array_max-array_min)+array_min
    return array

def NormStd(array,dim):
    array_mean = np.nanmean(array,dim)
    array_std = np.nanstd(array,dim)
    array_nor = (array-array_mean)/array_std
    return array_nor,array_mean,array_std
    
def invNormStd(array_nor,array_mean,array_std):
    array = array_nor*array_std+array_mean
    return array 


###############################################################################

###############################################################################
# miss data
def MissData(array):
    array[np.where(np.isnan(array))] = 0
    return array

# negetive value
def NegeData(array):
    array[np.where(array<0)] = 0
    return array
###############################################################################

###############################################################################
# preprocess
def data_preprocessing(k,X,time_dim,geo_dim,prcp_dim,is_prcp,is_geos):
    '''
        X: the inputs to be preprocessed
           could be predictors or predictands:[samples,lat,lon,vars]
        time_dim: the dimension to be averaged
        geo_dim: the number of geo-variables
        is_prcp: whether use Scaler
        is_geos: whether has geo-variables in predictors
    '''
    X = MissData(X)
    if is_prcp:
        X = Scaler(k,X)
    else:
        if prcp_dim>0:
            X[:,:,:,prcp_dim] = Scaler(k,X[:,:,:,prcp_dim])
        
    if is_geos: 
        X_geos = X[:,:,:,-geo_dim:]
        X_geos_nor = np.zeros(X_geos.shape)*np.nan
        for ivar in range(geo_dim):
            geo_max = np.nanmax(X_geos[:,:,:,ivar])
            geo_min = np.nanmin(X_geos[:,:,:,ivar])
            X_geos_nor[:,:,:,ivar] = (X_geos[:,:,:,ivar]-geo_min)/(geo_max-geo_min)
        
    # X = pad_to_shape(X) 
    
    X,X_max,X_min = NormMaxMin(X,time_dim)
    
    if is_geos:
        X[:,:,:,-geo_dim:] = X_geos_nor
        
    X = MissData(X)
    return X,X_max,X_min

# postprocess
def data_postprocessing(k,X,X_max,X_min,is_prcp):
    '''
        only for precipitaiton variable
    '''
    X = invNormMaxMin(X,X_max,X_min)
    # X = pred_to_rad(X)   
    X = MissData(X)
    if is_prcp:
        X = invScaler(k,X)
        X = NegeData(X)
    return X
###############################################################################

###############################################################################
# data preprocessing
geo_dim = 3
k = 0.01
Xtrain,X_max,X_min = data_preprocessing(k,PT_vars_Train,time_dim=0,geo_dim=geo_dim,prcp_dim=NWP_fcst_ids,is_prcp=False,is_geos=True)
Ytrain,Y_max,Y_min = data_preprocessing(k,GT_prep_Train,time_dim=0,geo_dim=0,prcp_dim=0,is_prcp=True,is_geos=False)
height,width,channels = Xtrain.shape[1],Xtrain.shape[2],Xtrain.shape[3]
del PT_vars_Train,GT_prep_Train

# test data preprocessing
PT_vars_Test_temp = copy.deepcopy(PT_vars_Test)
PT_vars_Test_temp[:,:,:,NWP_fcst_ids:NWP_fcst_ids+1] = Scaler(k, PT_vars_Test[:,:,:,NWP_fcst_ids:NWP_fcst_ids+1])
Xtest = (PT_vars_Test_temp-X_min)/(X_max-X_min)
num_test = Xtest.shape[0]
Xtest[:,:,:,-geo_dim:] = Xtrain[:num_test,:,:,-geo_dim:]
Xtest = MissData(Xtest)
del PT_vars_Test,GT_prep_Test,PT_vars_Test_temp
gc.collect()
###############################################################################

/tmp/ipykernel_26892/800362520.py:13: RuntimeWarning: invalid value encountered in true_divide
  array_nor = (array-array_min)/(array_max-array_min)
/tmp/ipykernel_26892/800362520.py:107: RuntimeWarning: invalid value encountered in true_divide
  Xtest = (PT_vars_Test_temp-X_min)/(X_max-X_min)


22

In [10]:
Xtrain_prcp = Xtrain[:,:,:,NWP_fcst_ids:NWP_fcst_ids+1]
Xtrain_max_prcp = X_max[:,:,NWP_fcst_ids:NWP_fcst_ids+1]
Xtrain_min_prcp = X_min[:,:,NWP_fcst_ids:NWP_fcst_ids+1]
Xtest_prcp = Xtest[:,:,:,NWP_fcst_ids:NWP_fcst_ids+1]
channels_prcp = 1

## Building model

In [11]:
def up_and_concate(down_layer, layer, data_format='channels_first'):
    if data_format == 'channels_first':
        in_channel = down_layer.get_shape().as_list()[1]
    else:
        in_channel = down_layer.get_shape().as_list()[3]

    # up = Conv2DTranspose(out_channel, [2, 2], strides=[2, 2])(down_layer)
    up = UpSampling2D(size=(2, 2), data_format=data_format)(down_layer)

    if data_format == 'channels_first':
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=1))
    else:
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))

    concate = my_concat([up, layer])

    return concate

def attention_up_and_concate(down_layer, layer, data_format='channels_first'):
    if data_format == 'channels_first':
        in_channel = down_layer.get_shape().as_list()[1]
    else:
        in_channel = down_layer.get_shape().as_list()[3]

    # up = Conv2DTranspose(out_channel, [2, 2], strides=[2, 2])(down_layer)
    up = UpSampling2D(size=(2, 2), data_format=data_format)(down_layer)

    layer = attention_block_2d(x=layer, g=up, inter_channel=in_channel // 4, data_format=data_format)

    if data_format == 'channels_first':
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=1))
    else:
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))

    concate = my_concat([up, layer])
    return concate

def attention_block_2d(x, g, inter_channel, data_format='channels_first'):
    # theta_x(?,g_height,g_width,inter_channel)

    theta_x = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(x)

    # phi_g(?,g_height,g_width,inter_channel)

    phi_g = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(g)

    # f(?,g_height,g_width,inter_channel)

    f = Activation('relu')(add([theta_x, phi_g]))

    # psi_f(?,g_height,g_width,1)

    psi_f = Conv2D(1, [1, 1], strides=[1, 1], data_format=data_format)(f)

    rate = Activation('sigmoid')(psi_f)

    # rate(?,x_height,x_width)

    # att_x(?,x_height,x_width,x_channel)

    att_x = multiply([x, rate])

    return att_x

def res_block(input_layer, out_n_filters, batch_normalization=False, kernel_size=[3, 3], stride=[1, 1],

              padding='same', data_format='channels_first'):
    if data_format == 'channels_first':
        input_n_filters = input_layer.get_shape().as_list()[1]
    else:
        input_n_filters = input_layer.get_shape().as_list()[3]

    layer = input_layer
    for i in range(2):
        layer = Conv2D(out_n_filters // 4, [1, 1], strides=stride, padding=padding, data_format=data_format)(layer)
        if batch_normalization:
            layer = BatchNormalization()(layer)
        layer = Activation('relu')(layer)
        layer = Conv2D(out_n_filters // 4, kernel_size, strides=stride, padding=padding, data_format=data_format)(layer)
        layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(layer)

    if out_n_filters != input_n_filters:
        skip_layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(
            input_layer)
    else:
        skip_layer = input_layer
    out_layer = add([layer, skip_layer])
    return out_layer

def rec_res_block(input_layer, out_n_filters, batch_normalization=False, kernel_size=[3, 3], stride=[1, 1],

                  padding='same', data_format='channels_first'):
    if data_format == 'channels_first':
        input_n_filters = input_layer.get_shape().as_list()[1]
    else:
        input_n_filters = input_layer.get_shape().as_list()[3]

    if out_n_filters != input_n_filters:
        skip_layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(
            input_layer)
    else:
        skip_layer = input_layer

    layer = skip_layer
    for j in range(2):

        for i in range(2):
            if i == 0:

                layer1 = Conv2D(out_n_filters, kernel_size, strides=stride, padding=padding, data_format=data_format)(
                    layer)
                if batch_normalization:
                    layer1 = BatchNormalization()(layer1)
                layer1 = Activation('relu')(layer1)
            layer1 = Conv2D(out_n_filters, kernel_size, strides=stride, padding=padding, data_format=data_format)(
                add([layer1, layer]))
            if batch_normalization:
                layer1 = BatchNormalization()(layer1)
            layer1 = Activation('relu')(layer1)
        layer = layer1

    out_layer = add([layer, skip_layer])
    return out_layer


In [12]:
Input_SHAPE = (height,width,channels_prcp)
Output_SHAPE = (height,width,1)

In [13]:
# Create the generator.
def generator_unet(n_label = 1, depth=4, features=64, \
                data_format='channels_last', mode='regression', name=''):
    
    inputs = Input(shape=Input_SHAPE)
    x = inputs
    skips = []
    for i in range(depth):
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        # x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format= data_format)(x)
        features = features * 2

    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
    x = Dropout(0.2)(x)
    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    for i in reversed(range(depth)):
        features = features // 2
        # attention_up_and_concate(x,[skips[i])
        x = UpSampling2D(size=(2, 2), data_format=data_format)(x)
        # print('skips[{0}] shape is: {1}'.format(i, skips[i].shape))
        # print('x shape is: {}'.format(x.shape))
        x = concatenate([skips[i], x], axis=-1)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        # x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    conv = Conv2D(n_label, (1, 1), padding='same', data_format=data_format)(x)
    if mode == 'regression':
        outputs = core.Activation('linear')(conv)
    elif mode == 'segmentation':
        outputs = core.Activation('sigmoid')(conv)
    model = Model(inputs=inputs, outputs=outputs)
    # tf.summary.histogram('outputs',outputs)

    return model, outputs


In [14]:
# Create the discriminator.
def discriminator(depth=4, features=64, \
                data_format='channels_last', name=''):
    inputs = Input(shape=Output_SHAPE)
    x = inputs
    for i in range(depth):
        x = Conv2D(features, (3, 3), (2, 2), activation='relu', padding='same', data_format=data_format)(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = LeakyReLU(alpha=0.2)(x)
        # x = Dropout(0.25)(x)
        features = features * 2
    x = Flatten()(x)
    outputs = Dense(1)(x) 
    model = Model(inputs=inputs, outputs=outputs)
    return model, outputs


In [15]:
g_model, g_out = generator_unet(depth=4, features=16)
d_model, d_out = discriminator(depth=4, features=16)

2022-04-25 09:29:58.221186: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-04-25 09:29:58.367641: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:61:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-04-25 09:29:58.368898: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:62:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-04-25 09:29:58.370072: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:89:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2

In [46]:
class WGAN(keras.Model):
    '''
    reference: https://keras.io/examples/generative/wgan_gp/
    '''
    def __init__(
        self,
        discriminator,
        generator,
        latent_dim,
        discriminator_extra_steps=3,
        gp_weight=10.0,
        grid_lambda=20.0,
    ):
        super(WGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_steps = discriminator_extra_steps
        self.gp_weight = gp_weight
        self.grid_lambda = grid_lambda

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(WGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn

    def gradient_penalty(self, batch_size, real_images, fake_images):
        """ Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        # Get the interpolated image
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated image.
            pred = self.discriminator(interpolated, training=True)

        # 2. Calculate the gradients w.r.t to this interpolated image.
        grads = gp_tape.gradient(pred, [interpolated])[0]
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, data):
        fake_predictors, real_images = data
        if isinstance(real_images, tuple):
            real_images = real_images[0]

        # Get the batch size
        batch_size = tf.shape(real_images)[0]

        # For each batch, we are going to perform the
        # following steps as laid out in the original paper:
        # 1. Train the generator and get the generator loss
        # 2. Train the discriminator and get the discriminator loss
        # 3. Calculate the gradient penalty
        # 4. Multiply this gradient penalty with a constant weight factor
        # 5. Add the gradient penalty to the discriminator loss
        # 6. Return the generator and discriminator losses as a loss dictionary

        # Train the discriminator first. The original paper recommends training
        # the discriminator for `x` more steps (typically 5) as compared to
        # one step of the generator. Here we will train it for 3 extra steps
        # as compared to 5 to reduce the training time.
        for i in range(self.d_steps):
            ### Get the latent vector
            # random_latent_vectors = tf.random.normal(
            #     shape=(batch_size, self.latent_dim)
            # )
            with tf.GradientTape() as tape:
                # Generate fake images from the latent vector
                fake_images = self.generator(fake_predictors, training=True)
                # Get the logits for the fake images
                fake_logits = self.discriminator(fake_images, training=True)
                # Get the logits for the real images
                real_logits = self.discriminator(real_images, training=True)

                # Calculate the discriminator loss using the fake and real image logits
                d_cost = self.d_loss_fn(real_img=real_logits, fake_img=fake_logits)
                # Calculate the gradient penalty
                gp = self.gradient_penalty(batch_size, real_images, fake_images)
                # Add the gradient penalty to the original discriminator loss
                d_loss = d_cost + gp * self.gp_weight

            # Get the gradients w.r.t the discriminator loss
            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            # Update the weights of the discriminator using the discriminator optimizer
            self.d_optimizer.apply_gradients(
                zip(d_gradient, self.discriminator.trainable_variables)
            )

        # Train the generator
        # Get the latent vector
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        with tf.GradientTape() as tape:
            # Generate fake images using the generator
            generated_images = self.generator(fake_predictors, training=True)
            # Get the discriminator logits for fake images
            gen_img_logits = self.discriminator(generated_images, training=True)
            # Calculate the generator loss
            g_loss = self.g_loss_fn(real_img=real_logits, fake_img=gen_img_logits, 
                                   grid_lambda = self.grid_lambda)
            

        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )
        return {"d_loss": d_loss, "g_loss": g_loss}


In [17]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, fake_predictors, num_img=6, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.fake_predictors = fake_predictors

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(self.fake_predictors)

        for i in range(self.num_img):
            img = generated_images[i].numpy()
            img = keras.preprocessing.image.array_to_img(img)
            img.save("generated_img_{i}_{epoch}.png".format(i=i, epoch=epoch))


In [58]:
# Instantiate the optimizer for both networks
# (learning_rate=0.0002, beta_1=0.5 are recommended)
generator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)
discriminator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)

# Define the loss functions for the discriminator,
# which should be (fake_loss - real_loss).
# We will add the gradient penalty later to this loss function.
def discriminator_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return fake_loss - real_loss

# Define the loss functions for the generator.
def generator_loss(real_img, fake_img, grid_lambda):
    """Grid cell regularizer.
    Args:
      generated_samples: Tensor of size [batch_size, h,w, 1].
      batch_targets: Tensor of size [batch_size, h,w, 1].
    Returns:
      loss: A tensor of shape [batch_size].
    """
    weights = tf.clip_by_value(real_img, 0.0, 24.0)
    grid_cell_reg = tf.reduce_mean(tf.abs(fake_img - real_img) * weights)
    return -tf.reduce_mean(fake_img) + grid_lambda * grid_cell_reg

# Set the number of epochs for trainining.
NOISE_DIM = 128

# # Instantiate the customer `GANMonitor` Keras callback.
# cbk = GANMonitor(tf.Variable(Xtrain), num_img=3, latent_dim=NOISE_DIM)

# Instantiate the WGAN model.
wgan = WGAN(
    discriminator=d_model,
    generator=g_model,
    latent_dim=NOISE_DIM,
    discriminator_extra_steps=3,
    grid_lambda = 1,
)

# Compile the WGAN model.
wgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
    g_loss_fn=generator_loss,
    d_loss_fn=discriminator_loss,
)


In [ ]:
# Start training the model.
BATCH_SIZE = 4
EPOCHS = 8
wgan.fit(Xtrain_prcp, Ytrain, 
         batch_size=BATCH_SIZE, epochs=EPOCHS) #, callbacks=[cbk]

Epoch 1/8
355/355 [==============================] - 24s 46ms/step - d_loss: 1042.0045 - g_loss: 426.7044
Epoch 2/8
 93/355 [======>.......................] - ETA: 11s - d_loss: 94.1263 - g_loss: 623.8875

In [56]:
trained_gen = wgan.generator
Ftest_nor = trained_gen.predict(Xtest_prcp)
Ftest = data_postprocessing(k,Ftest_nor,Y_max,Y_min,is_prcp=True)

In [57]:
evalute_fcst(Ytest,Ftest)

max of ref is 107.0
max of fcst is 25.776393140631328
rmse of fcst is 1.0429215234875673
csi of fcst is [[3.97083863e-03]
 [9.22215007e-05]
 [0.00000000e+00]
 [0.00000000e+00]]
evaluation costs 4.159373760223389 s


In [30]:
def RMSE(obs, sim):
    obs = obs.flatten()
    sim = sim.flatten()

    return np.sqrt(np.nanmean((obs - sim) ** 2))

def prep_clf(obs, sim, threshold=0.1):

    obs = np.where(obs >= threshold, 1, 0)
    sim = np.where(sim >= threshold, 1, 0)

    # True positive (TP)
    hits = np.sum((obs == 1) & (sim == 1))

    # False negative (FN)
    misses = np.sum((obs == 1) & (sim == 0))

    # False positive (FP)
    falsealarms = np.sum((obs == 0) & (sim == 1))

    # True negative (TN)
    correctnegatives = np.sum((obs == 0) & (sim == 0))

    return hits, misses, falsealarms, correctnegatives

def CSI(obs, sim, threshold=0.1):
    """
    CSI - critical success index
    details in the paper:
    Woo, W., & Wong, W. (2017).
    Operational Application of Optical Flow Techniques to Radar-Based
    Rainfall Nowcasting.
    Atmosphere, 8(3), 48. https://doi.org/10.3390/atmos8030048
    Args:
        obs (numpy.ndarray): observations
        sim (numpy.ndarray): simulations
        threshold (float)  : threshold for rainfall values binaryzation
                             (rain/no rain)
    Returns:
        float: CSI value
    """
    hits, misses, falsealarms, correctnegatives = prep_clf(obs=obs, sim=sim,
                                                           threshold=threshold)

    if (hits + misses + falsealarms)!=0:
        return hits / (hits + misses + falsealarms)
    else:
        return np.nan

def FAR(obs, sim, threshold=0.1):
    '''
    FAR - false alarm rate
    details in the paper:
    Woo, W., & Wong, W. (2017).
    Operational Application of Optical Flow Techniques to Radar-Based
    Rainfall Nowcasting.
    Atmosphere, 8(3), 48. https://doi.org/10.3390/atmos8030048
    Args:
        obs (numpy.ndarray): observations
        sim (numpy.ndarray): simulations
        threshold (float)  : threshold for rainfall values binaryzation
                             (rain/no rain)
    Returns:
        float: FAR value
    '''
    hits, misses, falsealarms, correctnegatives = prep_clf(obs=obs, sim=sim,
                                                           threshold=threshold)
    if (hits + falsealarms)!=0:
        return falsealarms / (hits + falsealarms)
    else:
        return np.nan

def POD(obs, sim, threshold=0.1):
    '''
    POD - probability of detection
    details in the paper:
    Woo, W., & Wong, W. (2017).
    Operational Application of Optical Flow Techniques to Radar-Based
    Rainfall Nowcasting.
    Atmosphere, 8(3), 48. https://doi.org/10.3390/atmos8030048
    Args:
        obs (numpy.ndarray): observations
        sim (numpy.ndarray): simulations
        threshold (float)  : threshold for rainfall values binaryzation
                             (rain/no rain)
    Returns:
        float: POD value
    '''
    hits, misses, falsealarms, correctnegatives = prep_clf(obs=obs, sim=sim,
                                                           threshold=threshold)
    if (hits + misses)!=0:
        return hits / (hits + misses)
    else:
        return np.nan

def ETS(obs, sim, threshold=0.1):
    '''
    ETS - Equitable Threat Score
    details in the paper:
    Winterrath, T., & Rosenow, W. (2007). A new module for the tracking of
    radar-derived precipitation with model-derived winds.
    Advances in Geosciences,10, 77–83. https://doi.org/10.5194/adgeo-10-77-2007

    Args:
        obs (numpy.ndarray): observations
        sim (numpy.ndarray): simulations
        threshold (float)  : threshold for rainfall values binaryzation
                             (rain/no rain)
    Returns:
        float: ETS value
    '''
    hits, misses, falsealarms, correctnegatives = prep_clf(obs=obs, sim=sim,
                                                           threshold=threshold)
    num = (hits + falsealarms) * (hits + misses)
    den = hits + misses + falsealarms + correctnegatives
    Dr = num / den

    if (hits + misses + falsealarms - Dr) != 0 :
        ETS = (hits - Dr) / (hits + misses + falsealarms - Dr)
    else:
        ETS = np.nan
    return ETS
    
def FSS(ref,fcst,TTT,R):
    '''
    compute FSS
    param ref: observation
    param fcst: forecast
    param R: influence radius
    param TTT: rain amount threshold, units:(m)
    the shape of input should be: [num_sample,lat,lon]  
    # here for a single file
    '''
    n_tim, n_lat, n_lon = ref.shape[0], ref.shape[1], ref.shape[2]
    ref_p = np.ones([n_lat, n_lon])*np.nan
    fcst_p = np.ones([n_lat, n_lon])*np.nan
    fbs = np.ones([n_tim])*np.nan
    fbs_worst = np.ones([n_tim])*np.nan
    for t in range(n_tim):
        print('>>>>>>>>>>>>>>>>>>>> {}'.format(t))
        for i in range(n_lat):
            for j in range(n_lon):            
                if ~np.isnan(ref[0,i,j]):
                    ref_box = ref[t,np.max([0,i-R]):np.min([n_lat,i+R+1]),np.max([0,j-R]):np.min([n_lon,j+R+1])]
                    fcst_box = fcst[t,np.max([0,i-R]):np.min([n_lat,i+R+1]),np.max([0,j-R]):np.min([n_lon,j+R+1])]
                    ref_p[i,j] = np.array(ref_box)[np.array(ref_box)>=TTT].size / (ref_box.size)
                    fcst_p[i,j] = np.array(fcst_box)[np.array(fcst_box)>=TTT].size / (ref_box.size)         

        fbs[t] = np.nanmean((ref_p-fcst_p)**2)
        fbs_worst[t] = np.nanmean(ref_p**2) + np.nanmean(fcst_p**2)
    fss = 1-np.nanmean(fbs)/np.nanmean(fbs_worst)
    return fss


In [31]:
def evalute_fcst(ref,fcst):
    time0 = time.time()
    print('max of ref is {}'.format(np.nanmax(ref)))
    print('max of fcst is {}'.format(np.nanmax(fcst)))

    rmse = RMSE(ref,fcst)
    print('rmse of fcst is {}'.format(rmse))

    csi_fcst = np.ones([4,1])*np.nan
    csi_fcst[0] = CSI(ref,fcst,0.1)
    csi_fcst[1] = CSI(ref,fcst,5)
    csi_fcst[2] = CSI(ref,fcst,10)
    csi_fcst[3] = CSI(ref,fcst,20)
    print('csi of fcst is {}'.format(csi_fcst))

    # fss_fcst = FSS(ref,fcst,20,3)
    # print('fss of fcst is {}'.format(fss_fcst))

    print('evaluation costs {} s'.format(time.time()-time0))

In [32]:
evalute_fcst(Ytest,Ftest)

max of ref is 107.0
max of fcst is 3.1851418458863976
rmse of fcst is 1.0392236644254458
csi of fcst is [[0.00700247]
 [0.        ]
 [0.        ]
 [0.        ]]
evaluation costs 4.0153467655181885 s
